In [76]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse

import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import heapq as hq

from itertools import chain

#import networkx as nx
#from networkx.drawing.nx_pydot import graphviz_layout
#import matplotlib.pyplot as plt

import numpy as np
import math

from datetime import datetime


In [68]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [69]:
opt_points_sets = [40, 100, 1_000, 5_000]
opt_dens = [0.1, 0.3, 0.7]

num_points = opt_points_sets[2] 
num_sets = num_points
density = opt_dens[1]

x = make_set_covering_problem(num_points, num_sets, density)

SETS=np.zeros(shape=(num_sets,num_points),dtype=bool)
for s,p in product(range(num_sets),range(num_points)):
    SETS[s,p]=x[s,p]

x=[[True,True,False,False],[True,False,True,False]]
print(np.logical_or.reduce(x,axis=0))


[ True  True  True False]


In [167]:
def fitness1(state):
    cost = np.count_nonzero(state)
    valid = np.all(
        np.logical_or.reduce(
            np.array([SETS[i,:] for i, t in enumerate(state) if np.any(t)]), axis=0
        )
    )
    return valid, -cost

def fitness2(state):
    cost = np.count_nonzero(state)
    valid = np.count_nonzero(
        np.logical_or.reduce(
            np.array([SETS[i,:] for i, t in enumerate(state) if np.any(t)]), axis=0
        )
    )
    return valid, -cost




def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i,:] for i in range(num_sets) if state[i]],
        np.array([False for _ in range(num_points)]),
    )


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = num_points - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

tot_step = 1000
step=0

offset_quadratic = tot_step/10
num_quadratic = tot_step+offset_quadratic

def fitness3(state):#the ground changes with steps
    
    cost = np.count_nonzero(state)
    valid = np.count_nonzero(
        np.logical_or.reduce(
            np.array([SETS[i,:] for i, t in enumerate(state) if np.any(t)]), axis=0
        )
    )
   
    return valid-(cost*((tot_step/(step+1))**2-1)),-cost
    

def fitness4(state):#the ground changes with steps, not usable with best_tweak since is expensive
    
    cost = np.count_nonzero(state)
    valid = np.count_nonzero(
        np.logical_or.reduce(
            np.array([SETS[i,:] for i, t in enumerate(state) if np.any(t)]), axis=0
        )
    )
    return valid-(cost*10*(((num_quadratic)/(step+offset_quadratic+1))**2-1)),-cost
    

def fitness5(state):#the ground changes with steps, not usable with best_tweak since is expensive
    
    cost = np.count_nonzero(state)
    valid = np.count_nonzero(
        np.logical_or.reduce(
            np.array([SETS[i,:] for i, t in enumerate(state) if np.any(t)]), axis=0
        )
    )
    return valid-(cost*10*(((num_quadratic)/(step+offset_quadratic+1))**2-1)+cost),-cost
    


fitness = fitness2


In [146]:
def tweak0(state):
    new_state = np.copy(state)
    index = randint(0, num_sets - 1)
    new_state[index] = not new_state[index]
    return new_state

n_tweak=int(num_points*math.sqrt(density))

def tweak1(state):
    seed(datetime.now().timestamp())
    new_state = np.copy(state)
    past_index = [-1]
    #magn=np.log10(tot_step)
    #n_tweak = (magn-np.log10(step+1))//2
    #n_tweak=max(n_tweak,1)
    
    for _ in range(int(n_tweak)):
        index = -1
        while index in past_index:
            width = ((step//10)*num_sets/10)%(num_sets), (((step//10)+1)*num_sets/10)%(num_sets)
            a=min(width[0],width[1])
            b=max(width[0],width[1])
            index = randint(a,b)
        past_index.append(index)
   
    for index in past_index:
        if index != -1:
            new_state[index] = not new_state[index]

    return new_state


count=np.count_nonzero(SETS,axis=1)
threshold=12
#print(count)

def get_best_candidates(state):
    
    choice_v=[]
    for i,n_tiles in enumerate(count):
       
        if n_tiles>=threshold:
            choice_v.append(i)

    return np.array(choice_v)

def tweak2(state):
    new_state = np.copy(state)
    past_index = []
    choice_v = get_best_candidates(new_state)
    for _ in range(n_tweak):
        index = np.random.choice(choice_v)
        while index in past_index:
            index = np.random.choice(choice_v)
        past_index.append(index)
    
    #print("tw") 
    for index in past_index:
        #print(index)
        new_state[index] = not new_state[index]

    return new_state


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i,:] for i in range(num_sets) if state[i]],
        np.array([False for _ in range(num_points)]),
    )


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = num_points - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def tweak3(state):
    sliding_width = int(num_sets/(np.log2(step+1)+1))
    
    interval = [(step*sliding_width)%num_sets, ((step+1)*sliding_width)%num_sets]
    if interval[0]>interval[1]:
        interval[0],interval[1]=interval[1],interval[0]

    new_state = np.copy(state)
    index = randint(interval[0],interval[1])
    new_state[index] = not new_state[index]
    return new_state

def best_tweak(state):#too slow: too many tweak to keep in account
    new_state = np.copy(state)
    max_fitness = -10000000,-10000000
    best_tweak = -1
    for index in range(0, num_sets):
        new_state[index] = not new_state[index]
        if (fitness(new_state)>max_fitness):
            max_fitness=fitness(new_state)
            best_tweak=index
        new_state[index] = not new_state[index]

    if best_tweak != -1:
        new_state[best_tweak] = not new_state[best_tweak]

    return new_state

def best_tweak2(state):#too slow: too many tweak to keep in account
    new_state = np.copy(state)
    max_fitness = -10000000,-10000000
    best_tweak = -1

    sliding_width = int(num_sets/(np.log2(step+1)+1))
    
    interval = [(step*sliding_width)%num_sets, ((step+1)*sliding_width)%num_sets]
    if interval[0]>interval[1]:
        interval[0],interval[1]=interval[1],interval[0]

    for index in range(interval[0],interval[1]):
        new_state[index] = not new_state[index]
        if (fitness(new_state)>max_fitness):
            max_fitness=fitness(new_state)
            best_tweak=index
        new_state[index] = not new_state[index]

    if best_tweak != -1:
        new_state[best_tweak] = not new_state[best_tweak]

    return new_state


    

tweak = tweak3


In [72]:
#generate starting point from greedy, check with A*



SETS = [SETS[i] for i in range(num_sets)]

State = namedtuple('State', ['taken', 'not_taken'])

def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(num_points)])))

def get_priority(state):
    set_covered=reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(num_points)]))
    return num_points-np.count_nonzero(set_covered)

frontier = PriorityQueue()
initial_state = State([], [i for i in range(num_sets)])
frontier.put((0,initial_state))

counter=0
current_state = frontier.get()[1]
priority = num_points

while priority > 0:
    counter += 1
    for i,action in enumerate(current_state.not_taken):
        
        new_taken = current_state.taken.copy()
        new_taken.append(action)
        new_not_taken = current_state.not_taken.copy()
        new_not_taken.pop(i)
        
        new_state = State(new_taken, new_not_taken)

        frontier.put((get_priority(new_state),new_state)) 
    
    priority_and_state = frontier.get()
    current_state=priority_and_state[1]
    priority=priority_and_state[0]

print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))

greedy_best = len(current_state.taken)
        


State(taken=[714, 404, 991, 572, 736, 951, 153, 212, 113, 6], not_taken=[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 

In [165]:
#if we take greedy as starting point we often have to random mutate more than 1 set

SETS = np.array(SETS)

def goal_check_matrix(state):
    return np.all(reduce(np.logical_or, [SETS[i,:] for i in range(num_sets) if state[i]], np.array([False for _ in range(num_points)])))


initial_state = np.zeros(shape=(num_sets),dtype=bool)
#for taken in current_state.taken:
#    initial_state[taken] = True

current_state = initial_state

seed(datetime.now().timestamp())

#if is a complete tweak use 50, if random use 1000
for step in range(tot_step):
    new_state = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state
        print(fitness(current_state))
        print(goal_check_matrix(current_state))
    if step % offset_quadratic == 0:
        print(offset_quadratic)
        
print(fitness(current_state))
print(goal_check_matrix(current_state))
print([i for i in range(num_sets) if current_state[i]])
#min 10 sol
#2m 44s best_tweak 2
#8m 37
#1_000 --> 11/12 21s
#10_000 --> 11   3m 24
#10_000 --> 11   3m 20s fitness5
#1_000 --> 11/12 wrong sometimes fitness5
#back to best_tweak problem
#2m 49s best_tweak 2 -> 10 fitness5
#2m 47s best_tweak 2 -> 11 fitness2 


100.0
(1.650520597555385, -1)
False
100.0
(144.7702845844221, -2)
False
(273.2793784273249, -3)
False
100.0
(432.9951325040563, -4)
False
(533.2849354803204, -5)
False
100.0
(656.5002704164413, -6)
False
100.0
(751.8296374159395, -7)
False
100.0
(811.6898461295043, -8)
False
100.0
(868.5220987803406, -9)
False
100.0
(937.9257576062257, -10)
True
100.0
100.0
(990.0, -10)
True
[93, 369, 582, 626, 711, 790, 824, 877, 944, 964]


In [844]:

#improved ordered A* that accounts for ordering and essentials sets

#computing essentials set
#idea from @Big_Pan on telegram group
#improve by forcing priority of list of set that do not contain essentials set to +INF

SETS = [SETS[i,:] for i in range(num_sets)]

def g(state):
    return len(state.taken)

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(num_points)]),
    )


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = num_points - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

#idea from @Big_Pan on telegram group
#improve by forcing priority of list of set that do not contain essentials set to +INF

#computing essentials set

necessary_sets=[]

for n,set in enumerate(SETS):
    NEW_SETS=tuple(SETS[i] for i in chain(range(n),range(n+1,num_sets)))

    other_set_covered=reduce(np.logical_or, [NEW_SETS[i] for i in range(num_sets-1)] , np.array([False for _ in range(num_points)]))
    set_still_not_covered=np.where(other_set_covered,False,True)
    #set_complete=np.all(reduce(np.logical_or,set_still_not_covered,set)) # optional if we check that goal()==true is sufficient np.any(set_still_not_covered)
    set_complete=np.any(set_still_not_covered)
    if set_complete:
        necessary_sets.append(n)

print(f"necessary set:{necessary_sets}")

useless_sets=[]

for n,set in enumerate(SETS):
    if not np.any(set):
        useless_sets.append(n)

print(f"useless set:{useless_sets}")

#those set are included by default and doesn't count in ordering check

def is_ordered(state):
    last_set=len(necessary_sets)
    for set in state.taken[last_set:]:
        if set<last_set:
            return False
        last_set=set
    return True

frontier = PriorityQueue()
initial_state = State([], [i for i in range(num_sets)])

for s in useless_sets:
    initial_state.not_taken.remove(s)

#updating initial state
for s in necessary_sets:
    initial_state.taken.append(s)
    initial_state.not_taken.remove(s)

#0 since first priority doesn't really count
frontier.put((0,(initial_state,num_points)))


counter=0
current_state = frontier.get()[1][0]
exp_dist_old = num_points

while not (exp_dist_old == 0):
    counter += 1
   
    for i,action in enumerate(current_state.not_taken):
        
        new_taken = current_state.taken.copy()
        new_taken.append(action)
        new_not_taken = current_state.not_taken.copy()
        new_not_taken.remove(action)
        
        new_state = State(new_taken, new_not_taken)

        if is_ordered(new_state):
            act_cost_g = g(new_state)
            exp_dist = h3(new_state)
            priority = act_cost_g+exp_dist

            frontier.put((priority, (new_state, exp_dist)))

    priority__state_dist_old = frontier.get()

    current_state_exp_dist_old=priority__state_dist_old[1]

    current_state=current_state_exp_dist_old[0]
    exp_dist_old=current_state_exp_dist_old[1]

    
    #priority=priority_and_state[0] #not necessary



print(current_state)
print(len(current_state.taken))
print(f"Solved in {counter:,} steps")
print(goal_check(current_state))

#30x60@0.08 comparison h3/h5
#    step   time           step   time
# h3 4044   1m 58s      h5 868    28s
# h3 ----   -------     h5 ----   ----
#60x60@0.3 comparison h3/h5
# h3 1846   1m 35s      h5 406    21s
# h3 935    49s         h5 122    8s
#100x100@0.3 comparison h3/h5
# h3 ----   >10m        h5 ----   >10m
#trying h6
# h6 1123   3m 9s --> we notice that each step becomes slower increasing SIZE of SETS and PROBLEM
# also h6 computation of distance is more expensive than h5 and h3
#60x60@0.3 comparison h3/h5/h6
# h6 526    37s         h5 1111   50s         h3 1271   1m 28s


necessary set:[]
useless set:[]


KeyboardInterrupt: 

In [169]:
#simulated annealing

SETS = np.array(SETS)

def goal_check_matrix(state):
    return np.all(reduce(np.logical_or, [SETS[i,:] for i in range(num_sets) if state[i]], np.array([False for _ in range(num_points)])))


initial_state = np.zeros(shape=(num_sets),dtype=bool)
#for taken in current_state.taken:
#    initial_state[taken] = True

current_state = initial_state

seed(datetime.now().timestamp())

max_covered = np.max([np.count_nonzero(SETS[i,:]) for i in range (num_sets)])

def sa_cond(new_state,current_state):

    fs=fitness(current_state)
    fs_1=fitness(new_state)

    if fs_1[0] > fs[0]:
        return True

    p = np.e**(-(fs[0]-fs_1[0])/(max_covered*(1-step/tot_step)**2))

    if random.random() > p  and fs_1[1]>fs[1]:
        return True

#if is a complete tweak use 50, if random use 1000
for step in range(tot_step):
    new_state = tweak(current_state)
    if sa_cond(current_state, new_state):
        current_state = new_state
        if(goal_check_matrix(current_state)):
            print(fitness(current_state))

        
print(fitness(current_state))
print(goal_check_matrix(current_state))
print([i for i in range(num_sets) if current_state[i]])



(1000, -19)
(1000, -20)
(1000, -21)
(1000, -22)
(1000, -23)
(1000, -24)
(1000, -25)
(1000, -26)
(1000, -27)
(1000, -28)
(1000, -29)
(1000, -30)
(1000, -31)
(1000, -32)
(1000, -33)
(1000, -34)
(1000, -35)
(1000, -36)
(1000, -37)
(1000, -38)
(1000, -39)
(1000, -40)
(1000, -41)
(1000, -42)
(1000, -43)
(1000, -44)
(1000, -45)
(1000, -46)
(1000, -47)
(1000, -48)
(1000, -49)
(1000, -50)
(1000, -51)
(1000, -52)
(1000, -53)
(1000, -54)
(1000, -55)
(1000, -56)
(1000, -57)
(1000, -58)
(1000, -59)
(1000, -60)
(1000, -61)
(1000, -62)
(1000, -63)
(1000, -64)
(1000, -65)
(1000, -66)
(1000, -67)
(1000, -68)
(1000, -69)
(1000, -70)
(1000, -71)
(1000, -72)
(1000, -73)
(1000, -74)
(1000, -75)
(1000, -76)
(1000, -77)
(1000, -78)
(1000, -79)
(1000, -80)
(1000, -81)
(1000, -82)
(1000, -83)
(1000, -84)
(1000, -85)
(1000, -86)
(1000, -87)
(1000, -88)
(1000, -89)
(1000, -90)
(1000, -91)
(1000, -92)
(1000, -93)
(1000, -94)
(1000, -95)
(1000, -96)
(1000, -97)
(1000, -98)
(1000, -99)
(1000, -100)
(1000, -101)
(1